# GoldMiner Financial Analytics Dashboard

## Executive Summary

This notebook provides comprehensive financial insights from transaction data stored in Parquet format. The analysis includes:

- **Summary Statistics**: Key financial metrics including mean spend, standard deviation, and account type breakdown
- **Time Series Analysis**: Monthly spending trends and patterns
- **Category Analysis**: Spending distribution across different categories
- **Merchant Analysis**: Top merchants by total spend
- **Urgency Analysis**: Distribution of transaction urgency levels
- **Anomaly Detection**: Identification and visualization of unusual transactions

All visualizations are interactive and can be filtered using the widgets below. Plots can be exported as PNG files for reports.

---

## 1. Setup and Configuration

Import required libraries and configure visualization settings. This notebook is designed to work fully offline with locally stored data.

In [ ]:
# Core data processing libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Visualization libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Interactive widgets
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display, HTML, clear_output

# Configure matplotlib for better visuals
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# Configure plotly for offline use
import plotly.io as pio
pio.renderers.default = 'notebook'

print("✓ Libraries imported successfully")
print(f"✓ Pandas version: {pd.__version__}")
print(f"✓ Notebook ready for analysis")

## 2. Data Loading

Load transaction data from Parquet files. Parquet format provides efficient storage and fast loading times.

In [ ]:
# Define data path
DATA_PATH = Path('data/processed/transactions.parquet')

# Load data
try:
    df = pd.read_parquet(DATA_PATH)
    print(f"✓ Successfully loaded {len(df):,} transactions")
    print(f"✓ Date range: {df['date'].min().strftime('%Y-%m-%d')} to {df['date'].max().strftime('%Y-%m-%d')}")
    print(f"✓ Columns: {', '.join(df.columns)}")
except FileNotFoundError:
    print("❌ Error: transactions.parquet not found!")
    print("Please ensure the data file exists at:", DATA_PATH)
    raise

# Display basic info
print("\n" + "="*80)
print("Data Preview:")
print("="*80)
df.head()

## 3. Summary Statistics

Key financial metrics providing an overview of spending patterns, account usage, and transaction characteristics.

In [ ]:
# Calculate summary statistics
total_transactions = len(df)
total_spend = df['amount'].sum()
mean_spend = df['amount'].mean()
median_spend = df['amount'].median()
std_spend = df['amount'].std()
min_spend = df['amount'].min()
max_spend = df['amount'].max()

# Credit vs Debit analysis
credit_transactions = df[df['account_type'] == 'Credit']
debit_transactions = df[df['account_type'] == 'Debit']

credit_count = len(credit_transactions)
debit_count = len(debit_transactions)
credit_total = credit_transactions['amount'].sum()
debit_total = debit_transactions['amount'].sum()
credit_debit_ratio = credit_total / debit_total if debit_total > 0 else 0

# Anomaly statistics
anomaly_count = df[df['anomaly_flag'] == 'anomaly'].shape[0]
anomaly_pct = (anomaly_count / total_transactions) * 100

# Display summary
print("="*80)
print("FINANCIAL SUMMARY STATISTICS")
print("="*80)
print(f"\nTransaction Overview:")
print(f"  • Total Transactions: {total_transactions:,}")
print(f"  • Total Spend: ${total_spend:,.2f}")
print(f"  • Mean Transaction: ${mean_spend:,.2f}")
print(f"  • Median Transaction: ${median_spend:,.2f}")
print(f"  • Std Deviation: ${std_spend:,.2f}")
print(f"  • Min Transaction: ${min_spend:,.2f}")
print(f"  • Max Transaction: ${max_spend:,.2f}")

print(f"\nAccount Type Analysis:")
print(f"  • Credit Card Transactions: {credit_count:,} ({credit_count/total_transactions*100:.1f}%)")
print(f"  • Credit Card Total: ${credit_total:,.2f}")
print(f"  • Debit Card Transactions: {debit_count:,} ({debit_count/total_transactions*100:.1f}%)")
print(f"  • Debit Card Total: ${debit_total:,.2f}")
print(f"  • Credit/Debit Spend Ratio: {credit_debit_ratio:.2f}:1")

print(f"\nAnomaly Detection:")
print(f"  • Anomalous Transactions: {anomaly_count:,} ({anomaly_pct:.2f}%)")
print(f"  • Normal Transactions: {total_transactions - anomaly_count:,}")

print("\n" + "="*80)

## 4. Interactive Filters

Use the widgets below to filter transactions by date range, category, account type, and urgency level. All visualizations will automatically update based on your selections.

In [ ]:
# Create filter widgets
min_date = df['date'].min().date()
max_date = df['date'].max().date()

# Date range picker
date_range = widgets.DatePicker(
    description='Start Date:',
    value=min_date,
    disabled=False
)

date_range_end = widgets.DatePicker(
    description='End Date:',
    value=max_date,
    disabled=False
)

# Category filter
categories = ['All'] + sorted(df['category'].unique().tolist())
category_filter = widgets.SelectMultiple(
    options=categories,
    value=['All'],
    description='Categories:',
    disabled=False,
    rows=6
)

# Account type filter
account_types = ['All'] + sorted(df['account_type'].unique().tolist())
account_filter = widgets.SelectMultiple(
    options=account_types,
    value=['All'],
    description='Account Type:',
    disabled=False
)

# Urgency filter
urgency_levels = ['All'] + sorted(df['urgency'].unique().tolist())
urgency_filter = widgets.SelectMultiple(
    options=urgency_levels,
    value=['All'],
    description='Urgency:',
    disabled=False
)

# Apply filters button
apply_button = widgets.Button(
    description='Apply Filters',
    button_style='success',
    tooltip='Click to apply filters and refresh visualizations',
    icon='check'
)

# Reset filters button
reset_button = widgets.Button(
    description='Reset Filters',
    button_style='warning',
    tooltip='Click to reset all filters',
    icon='refresh'
)

# Output widget for displaying filtered data info
filter_output = widgets.Output()

# Global variable to store filtered data
filtered_df = df.copy()

def apply_filters(b):
    """Apply selected filters to the dataframe."""
    global filtered_df
    
    with filter_output:
        clear_output()
        
        # Start with full dataset
        filtered_df = df.copy()
        
        # Apply date filter
        start_date = pd.Timestamp(date_range.value)
        end_date = pd.Timestamp(date_range_end.value)
        filtered_df = filtered_df[(filtered_df['date'] >= start_date) & (filtered_df['date'] <= end_date)]
        
        # Apply category filter
        if 'All' not in category_filter.value:
            filtered_df = filtered_df[filtered_df['category'].isin(category_filter.value)]
        
        # Apply account type filter
        if 'All' not in account_filter.value:
            filtered_df = filtered_df[filtered_df['account_type'].isin(account_filter.value)]
        
        # Apply urgency filter
        if 'All' not in urgency_filter.value:
            filtered_df = filtered_df[filtered_df['urgency'].isin(urgency_filter.value)]
        
        print(f"✓ Filters applied successfully!")
        print(f"✓ Filtered dataset contains {len(filtered_df):,} transactions")
        print(f"✓ Total filtered spend: ${filtered_df['amount'].sum():,.2f}")
        print(f"\nPlease run the visualization cells below to see updated charts.")

def reset_filters(b):
    """Reset all filters to default values."""
    global filtered_df
    
    with filter_output:
        clear_output()
        
        date_range.value = min_date
        date_range_end.value = max_date
        category_filter.value = ['All']
        account_filter.value = ['All']
        urgency_filter.value = ['All']
        filtered_df = df.copy()
        
        print("✓ All filters reset to default values")
        print(f"✓ Showing all {len(filtered_df):,} transactions")

# Attach click handlers
apply_button.on_click(apply_filters)
reset_button.on_click(reset_filters)

# Display filter UI
print("="*80)
print("INTERACTIVE FILTERS")
print("="*80)
print("\nSelect your filters and click 'Apply Filters' to update visualizations.\n")

filter_box = widgets.VBox([
    widgets.HBox([date_range, date_range_end]),
    widgets.HBox([category_filter, account_filter, urgency_filter]),
    widgets.HBox([apply_button, reset_button]),
    filter_output
])

display(filter_box)

## 5. Monthly Spend Timeline

Interactive line chart showing spending trends over time, aggregated by month. This helps identify seasonal patterns and spending trends.

In [ ]:
# Prepare monthly aggregation
monthly_data = filtered_df.copy()
monthly_data['year_month'] = monthly_data['date'].dt.to_period('M')
monthly_spend = monthly_data.groupby('year_month')['amount'].agg(['sum', 'count', 'mean']).reset_index()
monthly_spend['year_month'] = monthly_spend['year_month'].astype(str)

# Create interactive plotly chart
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=monthly_spend['year_month'],
    y=monthly_spend['sum'],
    mode='lines+markers',
    name='Total Spend',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8),
    hovertemplate='<b>%{x}</b><br>Total Spend: $%{y:,.2f}<extra></extra>'
))

fig.update_layout(
    title='Monthly Spending Timeline',
    xaxis_title='Month',
    yaxis_title='Total Spend ($)',
    hovermode='x unified',
    template='plotly_white',
    height=500,
    showlegend=True
)

fig.show()

# Print summary
print(f"\n📊 Monthly Spend Summary:")
print(f"  • Highest month: {monthly_spend.loc[monthly_spend['sum'].idxmax(), 'year_month']} (${monthly_spend['sum'].max():,.2f})")
print(f"  • Lowest month: {monthly_spend.loc[monthly_spend['sum'].idxmin(), 'year_month']} (${monthly_spend['sum'].min():,.2f})")
print(f"  • Average monthly spend: ${monthly_spend['sum'].mean():,.2f}")

## 6. Category Breakdown

Visual representation of spending distribution across different categories using both pie chart and treemap for comprehensive analysis.

In [ ]:
# Aggregate by category
category_spend = filtered_df.groupby('category')['amount'].sum().sort_values(ascending=False).reset_index()

# Create pie chart
fig_pie = px.pie(
    category_spend,
    values='amount',
    names='category',
    title='Spending by Category (Pie Chart)',
    hole=0.3,
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig_pie.update_traces(
    textposition='inside',
    textinfo='percent+label',
    hovertemplate='<b>%{label}</b><br>Amount: $%{value:,.2f}<br>Percentage: %{percent}<extra></extra>'
)

fig_pie.update_layout(height=500)
fig_pie.show()

# Create treemap
fig_tree = px.treemap(
    category_spend,
    path=['category'],
    values='amount',
    title='Spending by Category (Treemap)',
    color='amount',
    color_continuous_scale='Blues',
    hover_data={'amount': ':$,.2f'}
)

fig_tree.update_layout(height=500)
fig_tree.show()

# Print category summary
print(f"\n📊 Category Breakdown:")
for idx, row in category_spend.head(5).iterrows():
    pct = (row['amount'] / category_spend['amount'].sum()) * 100
    print(f"  • {row['category']}: ${row['amount']:,.2f} ({pct:.1f}%)")

## 7. Top 10 Merchants by Spend

Bar chart showing the merchants with the highest total spending. Useful for identifying primary vendors and subscription services.

In [ ]:
# Aggregate by merchant
merchant_spend = filtered_df.groupby('payee')['amount'].agg(['sum', 'count']).reset_index()
merchant_spend.columns = ['merchant', 'total_spend', 'transaction_count']
merchant_spend = merchant_spend.sort_values('total_spend', ascending=False).head(10)

# Create bar chart
fig = px.bar(
    merchant_spend,
    x='total_spend',
    y='merchant',
    orientation='h',
    title='Top 10 Merchants by Total Spend',
    labels={'total_spend': 'Total Spend ($)', 'merchant': 'Merchant'},
    text='total_spend',
    color='total_spend',
    color_continuous_scale='Viridis'
)

fig.update_traces(
    texttemplate='$%{text:,.2f}',
    textposition='outside',
    hovertemplate='<b>%{y}</b><br>Total Spend: $%{x:,.2f}<br>Transactions: %{customdata[0]}<extra></extra>',
    customdata=merchant_spend[['transaction_count']]
)

fig.update_layout(
    yaxis={'categoryorder': 'total ascending'},
    height=500,
    showlegend=False
)

fig.show()

# Print merchant summary
print(f"\n📊 Top Merchants Summary:")
for idx, row in merchant_spend.head(5).iterrows():
    avg_transaction = row['total_spend'] / row['transaction_count']
    print(f"  • {row['merchant']}: ${row['total_spend']:,.2f} ({row['transaction_count']} transactions, avg: ${avg_transaction:.2f})")

## 8. Urgency Level Distribution

Analysis of transaction urgency levels, showing both count distribution and spending patterns by urgency.

In [ ]:
# Aggregate by urgency
urgency_stats = filtered_df.groupby('urgency').agg({
    'amount': ['sum', 'count', 'mean']
}).reset_index()
urgency_stats.columns = ['urgency', 'total_spend', 'count', 'avg_spend']
urgency_stats = urgency_stats.sort_values('count', ascending=False)

# Create subplots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Transaction Count by Urgency', 'Total Spend by Urgency'),
    specs=[[{'type': 'pie'}, {'type': 'bar'}]]
)

# Add pie chart for count
fig.add_trace(
    go.Pie(
        labels=urgency_stats['urgency'],
        values=urgency_stats['count'],
        name='Count',
        marker_colors=px.colors.qualitative.Pastel
    ),
    row=1, col=1
)

# Add bar chart for spend
fig.add_trace(
    go.Bar(
        x=urgency_stats['urgency'],
        y=urgency_stats['total_spend'],
        name='Total Spend',
        marker_color='lightblue',
        text=urgency_stats['total_spend'],
        texttemplate='$%{text:,.0f}',
        textposition='outside'
    ),
    row=1, col=2
)

fig.update_layout(
    title_text='Urgency Level Distribution',
    height=500,
    showlegend=False
)

fig.show()

# Print urgency summary
print(f"\n📊 Urgency Level Summary:")
for idx, row in urgency_stats.iterrows():
    pct = (row['count'] / urgency_stats['count'].sum()) * 100
    print(f"  • {row['urgency'].capitalize()}: {row['count']} transactions ({pct:.1f}%), ${row['total_spend']:,.2f} total")

## 9. Anomaly Detection Timeline

Time series visualization highlighting anomalous transactions. Anomalies are marked in red for easy identification.

In [ ]:
# Prepare data for anomaly visualization
anomaly_data = filtered_df.copy()
anomaly_data['is_anomaly'] = anomaly_data['anomaly_flag'] == 'anomaly'

# Create scatter plot with normal and anomaly transactions
fig = go.Figure()

# Add normal transactions
normal_data = anomaly_data[~anomaly_data['is_anomaly']]
fig.add_trace(go.Scatter(
    x=normal_data['date'],
    y=normal_data['amount'],
    mode='markers',
    name='Normal Transactions',
    marker=dict(size=6, color='blue', opacity=0.5),
    hovertemplate='<b>Normal Transaction</b><br>Date: %{x}<br>Amount: $%{y:,.2f}<br>Category: %{customdata[0]}<br>Merchant: %{customdata[1]}<extra></extra>',
    customdata=normal_data[['category', 'payee']]
))

# Add anomaly transactions
anomaly_only = anomaly_data[anomaly_data['is_anomaly']]
if len(anomaly_only) > 0:
    fig.add_trace(go.Scatter(
        x=anomaly_only['date'],
        y=anomaly_only['amount'],
        mode='markers',
        name='Anomalies',
        marker=dict(
            size=12,
            color='red',
            symbol='diamond',
            line=dict(width=2, color='darkred')
        ),
        hovertemplate='<b>⚠️ ANOMALY</b><br>Date: %{x}<br>Amount: $%{y:,.2f}<br>Category: %{customdata[0]}<br>Merchant: %{customdata[1]}<extra></extra>',
        customdata=anomaly_only[['category', 'payee']]
    ))

fig.update_layout(
    title='Transaction Timeline with Anomaly Detection',
    xaxis_title='Date',
    yaxis_title='Transaction Amount ($)',
    hovermode='closest',
    template='plotly_white',
    height=600,
    showlegend=True
)

fig.show()

# Print anomaly summary
if len(anomaly_only) > 0:
    print(f"\n⚠️ Anomaly Detection Summary:")
    print(f"  • Total anomalies detected: {len(anomaly_only)}")
    print(f"  • Anomaly rate: {(len(anomaly_only)/len(anomaly_data))*100:.2f}%")
    print(f"  • Total anomalous spend: ${anomaly_only['amount'].sum():,.2f}")
    print(f"  • Average anomaly amount: ${anomaly_only['amount'].mean():,.2f}")
    print(f"\n  Top 5 Anomalous Transactions:")
    for idx, row in anomaly_only.nlargest(5, 'amount').iterrows():
        print(f"    • {row['date'].strftime('%Y-%m-%d')}: ${row['amount']:,.2f} at {row['payee']} ({row['category']})")
else:
    print(f"\n✓ No anomalies detected in the filtered dataset.")

## 10. Export Visualizations

Export all charts as PNG files for use in reports and presentations. Files will be saved to the `exports/` directory.

In [ ]:
import os

# Create exports directory if it doesn't exist
export_dir = Path('exports')
export_dir.mkdir(exist_ok=True)

print("Exporting visualizations...\n")

# Export monthly timeline
monthly_data = filtered_df.copy()
monthly_data['year_month'] = monthly_data['date'].dt.to_period('M')
monthly_spend = monthly_data.groupby('year_month')['amount'].sum().reset_index()
monthly_spend['year_month'] = monthly_spend['year_month'].astype(str)

fig1 = go.Figure()
fig1.add_trace(go.Scatter(
    x=monthly_spend['year_month'],
    y=monthly_spend['amount'],
    mode='lines+markers',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8)
))
fig1.update_layout(title='Monthly Spending Timeline', xaxis_title='Month', yaxis_title='Total Spend ($)', template='plotly_white')
fig1.write_image(str(export_dir / 'monthly_timeline.png'), width=1200, height=600)
print("✓ Exported: monthly_timeline.png")

# Export category breakdown
category_spend = filtered_df.groupby('category')['amount'].sum().reset_index()
fig2 = px.pie(category_spend, values='amount', names='category', title='Spending by Category')
fig2.write_image(str(export_dir / 'category_breakdown.png'), width=1200, height=600)
print("✓ Exported: category_breakdown.png")

# Export top merchants
merchant_spend = filtered_df.groupby('payee')['amount'].sum().reset_index()
merchant_spend = merchant_spend.sort_values('amount', ascending=False).head(10)
fig3 = px.bar(merchant_spend, x='amount', y='payee', orientation='h', title='Top 10 Merchants by Spend')
fig3.update_layout(yaxis={'categoryorder': 'total ascending'})
fig3.write_image(str(export_dir / 'top_merchants.png'), width=1200, height=600)
print("✓ Exported: top_merchants.png")

# Export urgency distribution
urgency_stats = filtered_df.groupby('urgency')['amount'].agg(['sum', 'count']).reset_index()
fig4 = px.pie(urgency_stats, values='count', names='urgency', title='Urgency Level Distribution')
fig4.write_image(str(export_dir / 'urgency_distribution.png'), width=1200, height=600)
print("✓ Exported: urgency_distribution.png")

# Export anomaly timeline
anomaly_data = filtered_df.copy()
anomaly_data['is_anomaly'] = anomaly_data['anomaly_flag'] == 'anomaly'
fig5 = go.Figure()
normal_data = anomaly_data[~anomaly_data['is_anomaly']]
fig5.add_trace(go.Scatter(x=normal_data['date'], y=normal_data['amount'], mode='markers', name='Normal', marker=dict(size=6, color='blue', opacity=0.5)))
anomaly_only = anomaly_data[anomaly_data['is_anomaly']]
if len(anomaly_only) > 0:
    fig5.add_trace(go.Scatter(x=anomaly_only['date'], y=anomaly_only['amount'], mode='markers', name='Anomalies', marker=dict(size=12, color='red', symbol='diamond')))
fig5.update_layout(title='Transaction Timeline with Anomaly Detection', xaxis_title='Date', yaxis_title='Amount ($)', template='plotly_white')
fig5.write_image(str(export_dir / 'anomaly_timeline.png'), width=1200, height=600)
print("✓ Exported: anomaly_timeline.png")

print(f"\n✓ All visualizations exported to '{export_dir}/' directory")
print(f"✓ Total files exported: 5")

## 11. Additional Analysis: Account Type Comparison

Detailed comparison between Credit and Debit card usage patterns.

In [ ]:
# Compare credit vs debit spending patterns
account_comparison = filtered_df.groupby(['account_type', 'category'])['amount'].sum().reset_index()

fig = px.bar(
    account_comparison,
    x='category',
    y='amount',
    color='account_type',
    title='Spending by Category: Credit vs Debit',
    labels={'amount': 'Total Spend ($)', 'category': 'Category'},
    barmode='group',
    color_discrete_map={'Credit': '#FF6B6B', 'Debit': '#4ECDC4'}
)

fig.update_layout(
    height=500,
    xaxis_tickangle=-45,
    template='plotly_white'
)

fig.show()

# Category preferences by account type
print("\n📊 Account Type Preferences:")
for account_type in filtered_df['account_type'].unique():
    account_data = filtered_df[filtered_df['account_type'] == account_type]
    top_category = account_data.groupby('category')['amount'].sum().idxmax()
    top_amount = account_data.groupby('category')['amount'].sum().max()
    print(f"  • {account_type} - Most spent on: {top_category} (${top_amount:,.2f})")

## 12. Conclusion

This notebook provides a comprehensive financial analytics dashboard with:
- Interactive filtering capabilities
- Multiple visualization types
- Anomaly detection and highlighting
- Export functionality for reports
- Fully offline operation

### Next Steps:
1. Apply filters to focus on specific time periods or categories
2. Export visualizations for executive presentations
3. Monitor anomalies for potential fraud or unusual spending
4. Use insights to optimize spending patterns

### Notes:
- All data is loaded from local Parquet files
- No internet connection required for operation
- Charts are interactive and can be zoomed/panned
- Filter changes require re-running visualization cells

---

*Generated by GoldMiner Analytics - Financial Intelligence Platform*